<a href="https://colab.research.google.com/github/bicks1/hughesintern/blob/main/csv_get_sequence_v0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install Modules

In [ ]:
from google.colab import drive, files
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
hg38 = "/content/drive/My Drive/Lab_share/Lab_member/SophiaBick/HughesIntern/Homo_sapiens.GRCh38.dna.toplevel.fa"

In [ ]:
!pip install pyfaidx

In [ ]:
from pyfaidx import Fasta

In [ ]:
import csv

In [ ]:
import pandas as pd

### Initialize Fasta Object

In [ ]:
genes = Fasta(hg38)

#### Practice

In [ ]:
print(genes)

Fasta("/content/drive/My Drive/Lab_share/Lab_member/SophiaBick/HughesIntern/Homo_sapiens.GRCh38.dna.toplevel.fa")


In [ ]:
genes.keys()

In [ ]:
# PLEKHN1 (+) from 966482 to 975865
genes['1'][966481:975865].seq

'AGGAGGCTGTGGACAGGGACCCAGACTTGCCGACCTGTACGACTCTGGCCATGGGGAACAGCCACTGTGTCCCTCAGGCCCCCAGGAGGCTCCGGGCCTCCTTCTCCAGAAAGCCCTCGCTGAAGGGAAACAGGTGAGCGGGGCGTGGGTGCGGCCACCTGGGCGCAGGGCTCCCCCACCCGCTCCGGGGCCAAGCCACGAGACCCCTTGCCTTGTCCCCAGAGAGGACAGCGCGCGGATGTCGGCCGGCCTGCCGGGCCCCGAGGCTGCTCGAAGCGGGGACGCCGCCGCCAACAAGCTCTTCCACTACATCCCGGGCACGGTGAGCGCGGCGTGCACGGTGGCTGTGGTCTGGGAGCGTGGCTCTGCCCGCGCGTGTGTGCCGTGTGTCCGTGCAGCTCAGGGTCTTCCCCTCGCCCCCGGGGCGTTCAGACCCCGGTAGGTGAGGAGCCGACGCTGACTGCCCCGCCCTGGGGAGCTCATCCGGCCGAAAGTCTGAGCGGAGAGCGAAACCGCGACGCAGGACTGAGTCAGTCGGGAGAGAGCTGGGCGGGGCGGGCCGCGGGCGGGGCCTCCCGGGACCTCCTCCCGCAGGGGACCAGCGCACGTGTCTCTGTCGCTGCAGTGGGGCTTGGGGTCGTGCCGGCAGGTAGTGTCACTTGAGAGTAAGGGGCCTCAGATGAGCAGCGAGCCAGCCGGTCACCTACACCGGCCTTTCTCCCTGGGTGTCCCGCAGGAAGTTGCCGCTGGGTATGGCCGCCTGGTCCACAGCAATCCCTCTGTGCACGCGCCGGCCTCCTCCCAGCTTGCCAAGGCCATGCAGCAAGGAGGTGTCTCCCAAGCACACACTAGGCCACCTCTCCGTCAAAGGAGCAGAAGACCACCCCCATGCCCACCCCCCCCCCAGCCCCTGTGGGACTAAGGGTCTTCACAGGGTAGATCCCAGCCCCTTTCAGATGTGTCTGGTGCTGGGATGAGGGAACAGGACCAGGAACCTGG

In [ ]:
# PERM1 (-) from 975198 to 982117 (want it 5' to 3' on its strand)
#genes['1'][975197:982117:-1].seq

#print("forward/", genes['1'][975197:982117])
print("reverse complement/", -genes['1'][975197:982117]) # want this way!


reverse complement/ ATGCCGACCCAGGACGGGCAGCTCAGACGCCCAGCTCGGCCGCCGGGACCCAGGTAGGACATGGTCAGCTGCTGGCTCCTGACCCAAGGGGCCCCAATGGTAACAACCCCCACAGAGGCTTCTCAAGAGACCAGAGAGGAGGGCGAAGGTCAGTGGGTGGGAAGCCACCATGCCCTGGCCTCCGTCTCAGGACCGTCCCCACCAAAGTGACTTCCCACCCCTGGGTCCAGAGGGGCTGACCTGAGGCCGAGGGCGGCCCCCTCTAGGGGTAGACCCGACAGCTGGCTAGGGTCTCCTTGCTTCTCTGAGACGGGGCAAAGGAAGGCACCCCGGGCAGCCGGGCAGGCCTGGATCGTGGTGCAGACCTCGAGGACGGCCCGAGGTGGCTGCTGGGTCAAGAGACCCACCGCAGCCTGCACCTCCTGCCCAGGCTCTCCGTGCTGACTTGGGCCAGCTCTGGCCAGCACCACCTGGTGAGCTCAGAGAATAGCCTGGCCCAGCCAAGGCGATCGGAAGGTCATCGCCGAGGGCTGGGCACCTGCTGGGCCCTGTAAACCTTTCCTGTGGCCGGCTGCCAGCTGGGGCGCACTGGGATGGACCAGGGCTGCCCACCCCCGAGGGCACTGGTCCCATCGGGTGGACCCTGAAAGGTGCTTTGCCGTGCAAAGACAGCTCTGGCAGAAGGTGGAGGAGGTGAGGCTGCCCCAGCTGAGGTGTGATGGGAAAGAGCATTCAAACCTGAGGACGGCTGGTTTTATAGCTTCACTGGTAGAAAATCATCCTTGATTCTAGGAATGTAACACCGCATCATCACACAGCTATCATTTGGGGTGCGTTGGGAACTAGGGTCATGGGAGCTGGGGATCTTCTAACTTGGAAATCGTCACCACTAGGTTTGAAGGAAGCGCGGCCCTCCCCTCGAGATGCTGTTCTCTTCCTTTCAGGGCATGGATGGAGCCCCGAGGCGGTGGGAGCTCCCA

In [ ]:
len(genes['1'])

248956422

### csv_to_fasta function

In [ ]:
def csv_to_fasta(csv_file, Fasta_obj, new_file):
  """
  :param csv_file: CSV from gff_gene_consensus with genes' chrom, name, coordinates, etc
  :param Fasta_obj: Fasta obj (pyfaidx) using fasta file with chromosome sequences
  :param new_file: Name for new fasta file with extracted gene sequences
  """
  with open(new_file, "w+") as fh:
    with open(csv_file, newline='') as csvfile:
      f_reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
      next(f_reader)
      for row in f_reader:

        chrom = row[0].split(',')[1]
        strand = row[0].split(',')[2]
        name = row[0].split(',')[3]
        id = row[0].split(',')[4]
        start = row[0].split(',')[5]
        end = row[0].split(',')[6]
        #print(id, name, chrom, strand, start, end)
        if strand == "+":
          seq = str(Fasta_obj[chrom][int(start)-1:int(end)])
        elif strand == "-":
          seq = str(-Fasta_obj[chrom][int(start)-1:int(end)])
        #print(seq[:20])


        fh.write(">{0} [acronym={1}] [chromosome={2}]".format(id, name, chrom))
        fh.write("\n{0}\n".format(seq))




In [ ]:
csvfile = "/content/drive/My Drive/Lab_share/Lab_member/SophiaBick/HughesIntern/gff_gene_consensus_gene.csv"

In [ ]:
csvfile2 = "/content/drive/My Drive/Lab_share/Lab_member/SophiaBick/HughesIntern/gff_gene_consensus_all.csv"

In [ ]:
csv_to_fasta(csvfile, genes, "csv_gene_seq_gene.fa")

In [ ]:
csv_to_fasta(csvfile2, genes, "csv_gene_seq_all.fa")

In [ ]:
# to download to local drive more simply, download gzip file
# !gzip "csv_gene_seq_gene.fa"  (3 minutes)
# !gzip "csv_gene_seq_all.fa"   (5 minutes)

In [ ]:
!cp "csv_gene_seq_gene.fa" "/content/drive/My Drive/Lab_share/Lab_member/SophiaBick/HughesIntern"

In [ ]:
!cp "csv_gene_seq_all.fa" "/content/drive/My Drive/Lab_share/Lab_member/SophiaBick/HughesIntern"

### representative transcript extraction

In [ ]:
all_transcripts = "/content/drive/My Drive/Lab_share/Lab_member/SophiaBick/HughesIntern/gff_gene_individual_all.csv"

In [ ]:
rep_a1 = "/content/drive/My Drive/Lab_share/Lab_member/SophiaBick/HughesIntern/gff_gene_representative_algorithm1.csv"

In [ ]:
df_all = pd.read_csv(all_transcripts)

<ipython-input-10-ed4121ea1069>:1: DtypeWarning: Columns (1,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df_all = pd.read_csv(all_transcripts)


In [ ]:
df_a1 = pd.read_csv(rep_a1)

In [ ]:
representative = df_a1["representative_transcript"]  # gather Series of representative transcripts via algorithm 1

In [ ]:
print(representative)
print(type(representative))

0        ENST00000641515
1        ENST00000426406
2        ENST00000332831
3        ENST00000618323
4        ENST00000477976
              ...       
62651    ENST00000420810
62652    ENST00000456738
62653    ENST00000435945
62654    ENST00000435741
62655    ENST00000431853
Name: representative_transcript, Length: 62656, dtype: object
<class 'pandas.core.series.Series'>


In [ ]:
df_filter = df_all.loc[df_all["transcript_id"].isin(representative)]  # filter csv file to only representative transcript info

In [ ]:
df_all["transcript_id"].isin(representative)

0          True
1          True
2          True
3         False
4         False
          ...  
252733     True
252734     True
252735     True
252736     True
252737     True
Name: transcript_id, Length: 252738, dtype: bool

In [ ]:
df_filter.reset_index(inplace=True)
df_filter.index += 1
df_filter.pop("index")
df_filter.pop("Unnamed: 0")

1             0
2             1
3             2
4            15
5            20
          ...  
62652    252733
62653    252734
62654    252735
62655    252736
62656    252737
Name: Unnamed: 0, Length: 62656, dtype: int64

In [ ]:
print(df_filter)

In [ ]:
len(df_filter) # 75 are left out because of ties

62656

In [ ]:
len(df_a1)

62656

In [ ]:
df_filter.to_csv("representative_transcript.csv")
df_filter.to_csv("/content/drive/My Drive/Lab_share/Lab_member/SophiaBick/HughesIntern/representative_transcript.csv")

In [ ]:
def trans_csv_to_fasta(csv_file, Fasta_obj, new_file):
  """
  :param csv_file: CSV from gff_gene_consensus with genes' chrom, name, coordinates, etc
  :param Fasta_obj: Fasta obj (pyfaidx) using fasta file with chromosome sequences
  :param new_file: Name for new fasta file with extracted gene sequences
  """
  with open(new_file, "w+") as fh:
    with open(csv_file, newline='') as csvfile:
      f_reader = csv.reader(csvfile)
      next(f_reader)
      for row in f_reader:  # for ea gene
        chrom = row[1]
        strand = row[2]
        gene_name = row[3]
        gene_id = row[4]
        transcript_name = row[7]
        transcript_id = row[8]
        exon_positions = row[12]
        seq = ''
        #print(gene_id, gene_name, chrom, strand)

        for exon in exon_positions.split(";"):  # ea exon chunk info
          #print(exon)
          coord = exon.split(":")[1]

          if strand == "+":
            #coord = exon.split(":")[1]  # exon's coordinates
            start_slice = coord.split('-')[0]  # exon slice locations
            end_slice = coord.split('-')[1]
            seq += str(Fasta_obj[chrom][int(start_slice)-1:int(end_slice)])

          elif strand == "-":  # file has '- strand' transcript coordinates 5' to 3' (big to small) BUT pyfaidx only supports small to big slice
            #coord = exon.split(":")[1]
            start_slice = coord.split('-')[1]  # start slice is actually exon's coordinate end and vice versa
            end_slice = coord.split('-')[0]  # since slicing only small to big
            seq += str(-Fasta_obj[chrom][int(start_slice)-1:int(end_slice)]) # reverse complement


        #print(seq[:20])



        fh.write(">{0} [acronym={1}] [chromosome={2}] [gene={3}]".format(transcript_id, transcript_name, chrom, gene_id))
        fh.write("\n{0}\n".format(seq))

In [ ]:
rep_trans = "/content/drive/My Drive/Lab_share/Lab_member/SophiaBick/HughesIntern/representative_transcript.csv"

In [ ]:
trans_csv_to_fasta(rep_trans, genes, "csv_rep_sequence.fa")

In [ ]:
!cp "csv_rep_sequence.fa" "/content/drive/My Drive/Lab_share/Lab_member/SophiaBick/HughesIntern"

In [ ]:
genes["1"][100000:100100]

>1:100001-100100
ACTAAGCACACAGAGAATAATGTCTAGAATCTGAGTGCCATGTTATCAAATTGTACTGAGACTCTTGCAGTCACACAGGCTGACATGTAAGCATCGCCAT

In [ ]:
genes["1"][100100:100000]

>1:100101-100101